## Moving Average Crossover

    In this notebook we will create a simple trading strategy using moving average crossover strategy. There are two kinds of signal we will be looking into for crossover strategy!
    a. Slow Signal: Moving average of relatively longer time period
    b. Fast Signal: Moving average of relatively shorter time period
    
**example:**

__1. M.A.(10 days) is fast signal when compared with M.A.(50 days)__

__2. M.A.(50 days) is fast signal when compared with M.A.(200 days)__   

        
    If fast signal cross over slow signal and is higher, we will buy the stock.
    If slow signal goes above fast signal we will sell the stock. 
    
    Note: If the fast signal is already above the slow signal, we won't enter the trade!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

##### Checking Apple stock OHLC, Adj Close and volume 

In [4]:
aapl = pd.read_csv('S&P500/2019_2020/AAPL_2019-01-01_2020-06-01.csv')
aapl.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-12-31,158.529999,159.360001,156.479996,157.740005,154.618546,35003500
1,2019-01-02,154.889999,158.850006,154.229996,157.919998,154.794983,37039700


##### Step 1: Adding fast and slow signal into the dataframe !


Note: We will be using 10 period and 50 period moving average as fast and slow signal

In [8]:
aapl['MA10'] = aapl['Close'].rolling(10).mean()
aapl['MA50'] = aapl['Close'].rolling(50).mean()
aapl.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume,MA10,MA50
0,2018-12-31,158.529999,159.360001,156.479996,157.740005,154.618546,35003500,NaN,NaN
1,2019-01-02,154.889999,158.850006,154.229996,157.919998,154.794983,37039700,NaN,NaN


##### Step 2: Dropping all NaN values rows from MA10 and MA50

In [9]:
aapl = aapl.dropna()
aapl.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume,MA10,MA50
49,2019-03-13,182.250000,183.300003,180.919998,181.710007,178.878098,31032500,176.095001,164.9172
50,2019-03-14,183.899994,184.100006,182.559998,183.729996,180.866608,23579500,177.153001,165.4370


##### Step 3 : Add a new column "Shares".

    If MA10>MA50, denote as 1 (long one share of stock), otherwise, denote as 0 (do nothing)

In [11]:
aapl['Shares'] = [1 if aapl.loc[ei, 'MA10']>aapl.loc[ei, 'MA50'] else 0 for ei in aapl.index]
aapl.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume,MA10,MA50,Shares
49,2019-03-13,182.250000,183.300003,180.919998,181.710007,178.878098,31032500,176.095001,164.9172,1
50,2019-03-14,183.899994,184.100006,182.559998,183.729996,180.866608,23579500,177.153001,165.4370,1


##### Step 4 : Restrucuring such that if shares == 1 then edit dataframe !!
    
    Note: If the fast signal is already above the slow signal, we will cut the dataframe such that the starting point is when Shares == 0

In [12]:
if aapl.iloc[0,-1] == 1:
    for i in range(len(aapl)):
        if aapl.iloc[i, -1] == 0:
            aapl = aapl.iloc[i:, :]
            break
            
aapl.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume,MA10,MA50,Shares
96,2019-05-20,183.520004,184.350006,180.279999,183.089996,180.930695,38612300,193.112999,195.6952,0
97,2019-05-21,185.220001,188.000000,184.699997,186.600006,184.399307,28364800,191.487000,195.8492,0


##### Step 5 : Calculating Profit !!
    Note: Profit is 0 if Shares = 0, i.e we aren't involved in any trading once Fast signal is below Slow signal!

In [15]:
aapl['Close1'] = aapl['Close'].shift(-1)
aapl['Profit'] = [aapl.loc[ei, 'Close1'] - aapl.loc[ei, 'Close'] if aapl.loc[ei, 'Shares']==1 else 0 for ei in aapl.index]
aapl.tail(2)

,Date,Open,High,Low,Close,Adj Close,Volume,MA10,MA50,Shares,Close1,Profit
354,2020-05-28,316.769989,323.440002,315.630005,318.250000,318.250000,33390200,315.341003,281.0924,1,317.940002,-0.309998
355,2020-05-29,319.250000,321.149994,316.470001,317.940002,317.940002,38399500,316.181003,282.5178,1,NaN,NaN


##### Step 6 : Calculating Wealth
    Wealth is cumsum of profit

In [18]:
aapl['wealth'] = aapl['Profit'].cumsum()
aapl.tail(3)

,Date,Open,High,Low,Close,Adj Close,Volume,MA10,MA50,Shares,Close1,Profit,wealth
353,2020-05-27,316.140015,318.709991,313.089996,318.109985,318.109985,28236300,314.281003,279.7846,1,318.250000,0.140015,107.439972
354,2020-05-28,316.769989,323.440002,315.630005,318.250000,318.250000,33390200,315.341003,281.0924,1,317.940002,-0.309998,107.129974
355,2020-05-29,319.250000,321.149994,316.470001,317.940002,317.940002,38399500,316.181003,282.5178,1,NaN,NaN,NaN


##### Step 7 : Calculating Buy price and net profit
    Note: Approximation: That we bought at the highest price on the day we purchased ! As it's daily price data and we don't know exactly at what price did we bought. So to be at safe side, we consider the highest price of purchase during that day. 

In [27]:
buy_price = list(aapl[aapl['Shares'] == 1]['High'])[0]
net_profit = aapl.loc[aapl.index[-2], 'wealth']
profit_ratio = (net_profit/buy_price) * 100
print('Buy price: ${}, Net Profit: ${}, Profit Ratio: {}%'.format(buy_price, net_profit, round(profit_ratio, 3)))


Buy price: $200.6100006103516, Net Profit: $107.12997436523449, Profit Ratio: 53.402%
